# 03-01 Pandas-Demo

## Hinweise zur Übung

Diese Demo schließt an die vorherige SQL-Übung an und zeigt, wie die wesentlichen Abfragemöglichkeiten statt auf eine SQL-Dataenbank auch in-memory mit Python-Dataframes und Funktionen des Pandas-Package umgesetzt werden können.

Caveat: Dataframes befinden sich komplett im Speicher ("in-memory"). Nachdem sie geladen sind, können Abfragen deshalb extrem schnell (ggf. auch schneller als auf einer Datenbank) durchgeführt werden. Allerdings können die Quelldataen ggf. größer als der verfügbare Speicher sein. In diesem Fall ist es sinnvoller, Vorselektionen auf Ebene der Datenbank durchzuführen und nur die tatsächlich benötigten Daten zu laden.

## Konfiguration des Notebooks

In [ ]:
# Ggf. fehlende Pakete installieren
!pip install --quiet ipython-sql pandas

In [ ]:
import os
import sys
import urllib.request
import gzip
import shutil
import pandas
%load_ext sql

In [ ]:
# Konfiguration
base_url_quellen   = "https://raw.githubusercontent.com/fau-lmi/lct-ehealth/main/08-Datenanalyse+Visualisierung/data"
base_url_reporting = "./"

In [ ]:
# SQlite-Datenbanken aus Github auf den Jupyter-Server herunterladen
urllib.request.urlretrieve(base_url_quellen + "/dwh/reporting.sqlite.gz", base_url_reporting + "reporting.sqlite.gz")

# Die Sqlite-Datenbank ist aufgrund ihrer Größe gezipped und muss vor der Nutzung noch entpackt werden
with gzip.open(base_url_reporting + "reporting.sqlite.gz", "rb") as f_in:
    with open(base_url_reporting + "reporting.sqlite", "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

In [ ]:
# Datenbankverbindung als Pfad (für das ETL) & iPython SQL (für die Abfragen) herstellen
db_path_reporting      = base_url_reporting + "reporting.sqlite"

db_url_reporting      = "sqlite:///" + db_path_reporting

%sql $db_url_reporting

## Laden von Daten aus der SQLite-Datenbank in Dataframes

Auch in diesem Abschnitt nutzen wir die SQLite-Datenbank mit dem Kurs-DWH. Wir laden die jeweils benötigten Daten über die bekannten SQL-Abfragen und überführen sie in einen Dataframe.

Bei Abfragen mit dem `%sql`-Kommando haben wir das Ergebnis bisher nur angezeigt. Wir ändern das hier, indem wir das Ergebnis in eine Variable speichern und anschließend mit der Methode `DataFrame()` in einen Dataframe konvertieren.

Die Überführung der Abfrageergebnisse in eine Variable kann nur bei "einzeiligen" SQL-Statements genutzt werden (`%sql`), nicht jedoch bei mehrzeiligen (`%%sql`). Wir schreiben das SQL-Statement daher zunächst in eine Variable und übergeben die Variable in ein einzeliges `%sql`-Statement. Die 3-fachen Anführungszeichen `"""` dienen in Python dazu, längere Texte mit Zeilenumbrüchen in einem Rutsch in eine Variable zu schreiben.

In [ ]:
# SQL-Statement zur Abfrage der kompletten Tabelle D_PATIENT in Variable speichern
sql = """
SELECT *
  FROM d_patient
"""

# Abfrage ausführen und Ergebnis in Variable resultset speichern
resultset = %sql $db_url_reporting $sql

# Resultset in Dataframe überführen
df_patient = resultset.DataFrame()

# Erste Zeilen des Dataframe ausgeben
df_patient.head()

In [ ]:
# Abfrage der Tabelle F_FAELLE und Ablage in Dataframe df_faelle
sql = """
SELECT *
  FROM f_faelle
"""
resultset = %sql $db_url_reporting $sql
df_faelle = resultset.DataFrame()
df_faelle.head()

In [ ]:
# Abfrage der Tabelle D_FALLART und Ablage in Dataframe df_fallart
sql = """
SELECT *
  FROM d_fallart
"""
resultset = %sql $db_url_reporting $sql
df_fallart = resultset.DataFrame()
df_fallart.head()

In [ ]:
# Abfrage der Tabelle D_DIAGNOSE und Ablage in Dataframe df_diagnose
sql = """
SELECT *
  FROM d_diagnose
"""
resultset = %sql $db_url_reporting $sql
df_diagnose = resultset.DataFrame()
df_diagnose.head()

## Einfache Abfragen auf Datafames

Dataframes können wie eine Datenbanktabelle mit verschiedenen Pandas-Methoden abgefragt werden, um z.B. nur bestimmte Spalten auszuwählen oder Zeilen nach verschiedenen Kriterien zu filtern.

### Spalten eines Dataframe selektieren

Die Spalten eines Dataframes können über die aus der Datenbank übernommenen Spaltennamen adressiert werden (entsprechend der `SELECT`-Klausel eines SQL-Statements). Die Notation ist hierbei `dataframename[['Spalte1'], ['Spalte2'], ...]`. Mit der `head()`-Methode wird wieder nur der Anfang des Ergebnissatzes ausgegeben.

In [ ]:
# Nur die Spalten patient_id & patient_nachname ausgeben
df_patient[["patient_id", "patient_nachname"]].head()

### Zeilen eines Dataframe selektieren

Vergleichbar zur `WHERE`-Klausel eines SQL-Statements kann in Pandas die `query()`-Methode genutzt werden, um ein oder mehrere Kriterien zur Selektion von Zeilen eines Dataframe anzuwenden. Spalten des Dataframe können dabei über ihren Namen direkt angesprochen werden.

In den Query-Kriterien können auch Variablen und (ausgewählte) Funktionen benutzt werden. Z.B. können Teilstringvergleiche u.a. durch Anfügen von `.str.startswith('text')` oder `.str.contains('')` durchgeführt werden.

Mehrere Kriterien können mit Bool'schen Operatoren verbunden werden, und zwar sowohl als Textkommandos (and, or, not) als auch Symbole (&, |).


In [ ]:
# Filterung auf männliche Patienten
df_patient.query("patient_geschlecht == 'M'").head()

In [ ]:
# Patienten selektieren, deren Geburtsdatum (über einen Stringvergleich) mit dem Jahr 1984 beginnt
df_patient.query("patient_gebdat.str.startswith('1984')").head()

In [ ]:
# Männliche Patienten selektieren, deren Geburtsdatum (über einen Stringvergleich) mit dem Jahr 1984 beginnt
df_patient.query("patient_geschlecht == 'M' and patient_gebdat.str.startswith('1984')").head()

### Eindeutige Datensätze auslesen

Analog zum `DISTINCT`-Keyword aus SQL gibt es in Pandas die `drop_duplicates()`-Methode, um über alle enthaltenen Spalten eindeutige Datensätze auszugeben.

Pandas erlaubt es, Methodenaufrufe hintereinander zu schreiben, so dass sie im Sinne einer Pipeline nacheinander ausgeführt werden (hier: erst `drop_duplicates()` und danach `head()`).

In [ ]:
# Nur die eindeutigen Ausprägungen der Spalte patient_geschlecht ausgeben
df_patient[['patient_geschlecht']].drop_duplicates().head()

## Gruppieren und aggregieren von Tabelleninhalten

Analog zu `GROUP BY`und den verschiedenen Aggregatfunktionen in SQL stellt Pandas die `groupby()`-Methode sowie die `agg()`-Methode bereit. Die Funktionen können wie in den vorherigen Beispielen als Pipeline hintereinander verkettet werden.

In der `groupby()`-Methode kann eine einzelne Spalte in Anführungsstrichen als Argument eingetragen werden (z.B. `groupby('Spalte')`, während mehrere Spalten als Array in eckige Klammern geschrieben werden müssen (z.B. `groupby(['Spalte1', 'Spalte2'])`).

In der `agg()`-Methode können die Aggregationen auf verschiedene Weisen angegeben werden. Die folgende Form erlaubt es, für mehrere Spalten verschiedene Aggregationen anzugeben und den daraus resultierenden Spalten explizite Namen zuzuweisen: `agg(aggregierte_spalte1=("quellspalte1", "aggregatfunktion"), aggregierte_spalte2=("quellspalte2", "aggregatfunktion"))`.

Beispiel:
* `dataframe.agg(erloes_sum=("erloes", "sum"), erloes_avg=("erloes", "mean"))

### Datensätze aggregieren

In [ ]:
# Patiententabelle nach Geschlecht gruppieren und die Anzahl der Datensätze zählen
df_patient.groupby("patient_geschlecht").agg(n=("patient_geschlecht", "size"))

In [ ]:
# Fälletabelle nach Fallart gruppieren und die Fallzahl, den Mittelwert der Liegedauer sowie Summe der Erlöse aggregieren
df_faelle.groupby("fallart_id").agg(fallzahl=("fallart_id", "size"), liegedauer_avg=("liegedauer_tage", "mean"), erloes_sum=("erloes_fallpauschale", "sum"))

### Filterung bei Aggregatfunktionen

Zur `HAVING`-Klausel aus SQL gibt es kein eigenes Äquivalent, stattdessen kann einfach die oben beschriebene `query()`-Methode in der Pipeline hinter die Aggregation gestellt werden.

Da die Pipelines bei vielen Verarbeitungsschritten unübersichtlich werden können, ist es möglich, die Schritte auf mehrere Zeilen aufzuteilen. Ans Ende der jeweils vorherigen Zeile muss ein Backslash (`\`) gestellt werden und die fortsetzenden Zeilen müssen eingerückt werden.

In [ ]:
# Fälletabelle nach Fallart gruppieren und die Fallzahl, den Mittelwert der Liegedauer sowie Summe der Erlöse aggregieren
# - Filterung nach der Aggregation auf Einträge mit mittlerer Liegedauer > 6 Tage
df_faelle.groupby("fallart_id") \
  .agg(fallzahl=("fallart_id", "size"), liegedauer_avg=("liegedauer_tage", "mean"), erloes_sum=("erloes_fallpauschale", "sum")) \
  .query("liegedauer_avg > 6")

## Zusammenführen mehrerer Dataframes

Das Gegenstück zur `JOIN`-Klausel in SQL ist die `merge()`-Methode in Pandas, die ebenfalls inner sowie left/right outer Joins über eine oder mehrere Spalten der einbezogenen Dataframes erlaubt.

Die `merge()`-Methode kann in einer Pipeline an den ersten ("linken") in den Join einbezogenen Dataframe angehängt werden und enthält als erstes Argument den "rechts" zu verbindenen Dataframe: `dataframe1.merge(dataframe2)`.

Mit dem `on´-Argument werden die Spalten festgelegt, die für die Verknüpfung der Dataframes eingesetzt werden sollen:
* `on='Spalte1'`: es soll nur über Spalte1 gejoined werden, die in beiden Dataframes identisch benannt ist
* `on=['Spalte1', 'Spalte2']`: es soll über Spalte1 und Spalte2 gejoined werden, die in beiden Dataframes identisch benannt sind
* `left_on=['LinkeSpalte1', 'LinkeSpalte2'], right_on=[: 'RechteSpalte1', 'RechteSpalte2']`: wenn die Spalten der Join-Kriterien in den beiden Dataframes unterschiedlich benannt sind, müssen sie separat in `left_on`und `right_on`-Argumenten in identischer Reihenfolge angegeben werden.

Identisch benannte Spalten, die als Join-Kriterien angegeben wurden, werden im Ergebnisdataframe zusammengeführt, während unterschiedlich benannte Spalten übernommen werden. Identisch benannte Spalten außerhalb der Join-Kriterien werden separat in den Ergebnisdataframe benommen und ggf. mit einem Suffix qualifiziert.

"Überflüssige" Spalten, die z.B. wegen Redundanzen zwischen den Tabellen im Ergebnisdataframe enthalten sind, können mit der `drop()`-Methode entfernt werden.



### INNER JOIN: Schnittmenge von Dataframes bilden



In [ ]:
# Spalten für Fall-ID und Fallart-ID aus dem Dataframe df_faelle selektieren und
# den Fallartbezeichner per merge() hinzufügen
df_faelle[["fall_id", "fallart_id"]].merge(df_fallart, on="fallart_id").head()

In [ ]:
# Diagnosebezeichner zu den Fällen ergänzen
df_faelle[["fall_id", "hauptdiagnose_snomed_id"]] \
  .merge(df_diagnose[["snomed_id", "snomed_name"]], left_on="hauptdiagnose_snomed_id", right_on="snomed_id") \
  .head()

In [ ]:
# Pipeline für Aggregation & Zählung der Fälle nach Fallart und Ergänzung der Fallartbezeichner
df_faelle.merge(df_fallart, on="fallart_id") \
  .groupby(["fallart_id", "fallart_name"]) \
  .agg({"fallart_id": "size"})

### OUTER JOINs: Datensätze einbeziehen, für die es keine Entsprechung in beiden Dataframes gibt

Mit dem `how`-Argument kann festgelegt werden, ob ein inner (default), left oder right outer join verwendet werden soll.

In [ ]:
# Fälle mit Bezeichnern für ihre Hauptdiagnosen ausgeben, aber Fälle ohne Hauptdiagnose beibehalten
df_faelle[["fall_id", "hauptdiagnose_snomed_id"]] \
  .merge(df_diagnose[["snomed_id", "snomed_name"]], left_on="hauptdiagnose_snomed_id", right_on="snomed_id", how="left") \
  .drop(columns="snomed_id") \
  .head()

### Mehrere  Dataframes "hintereinander" zusammenfügen

Neben dem "horizontalen" Verknüpfen von Dataframes mit der `merge()`-Methode können Dataframes auch mit der `concat()`-Funktion "hintereinander gestellt" werden. Die Spalten beides Datensätze müssen dafür identische Namen und Spalten haben.

Im Gegensatz zu den oben beschriebenen Methoden ist `concat()` eine eigenständige Funktion, der beide (oder auch mehrere) hintereinanderzusetzende Dataframes als Argument übergeben werden müssen, und keine Methode eines einzelnen Dataframe.

Im Gegensatz zum `UNION`-Statement in SQL werden hier beide Datensätze vollständig hintereinander gesetzt, unabhängig davon, ob es doppelte Datensätze gibt. Die `concat()`-Methode enspricht also einem `UNION ALL`-Statement in SQL. Falls eindeutige Datensätze benötigt werden, können mit der `drop_duplicates()`-Methode Zeilen mit identischen Inhalten zusammengeführt werden.

In [ ]:
# Zwei Datensätze mit Teilen des Diagnosehierarchie selektieren (Bronchitis und Diabetes)
df_bronchitis = df_diagnose.query("snomed_name.str.contains('bronchitis')")
df_diabetes   = df_diagnose.query("snomed_name.str.contains('diabetes')")

# Beide Datensätze mit concat() zusammenfügen
pandas.concat([df_bronchitis, df_diabetes])